## Plotly plot of an isosurface ##

An isosurface, F(x,y,z) = c, is discretized by a triangular mesh, extracted by the [Marching cubes algorithm](http://scikit-image.org/docs/dev/auto_examples/edges/plot_marching_cubes.html) from a volume given as a (M, N, P) array of doubles. 
The scikit image function, measure.marching_cubes(F, c) returns the vertices and simplices of the triangulated surface.


In [44]:
import numpy as np
import matplotlib.cm as cm
from skimage import measure

In [45]:
import plotly.plotly as py
from plotly.graph_objs import *
import plotly
plotly.offline.init_notebook_mode() 

The next three functions are devoted to plotting a trisurf, via Plotly:

In [46]:
def map_z2color(zval, colormap, vmin, vmax):
    #map the normalized value val to a corresponding color in the mpl colormap
    
    if vmin>=vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(zval-vmin)/float((vmax-vmin))#normalize val
    C=map(np.uint8, np.array(colormap(t)[:3])*255)
    #convert to a Plotly color code:
    return 'rgb'+str((C[0], C[1], C[2]))

In [47]:
def mpl_to_plotly(cmap, pl_entries):
    h=1.0/(pl_entries-1)
    pl_colorscale=[]
    for k in range(pl_entries):
        C=map(np.uint8, np.array(cmap(k*h)[:3])*255)
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
    return pl_colorscale

In [48]:
def plotly_trisurf(x, y, z, simplices, colormap=cm.RdBu, showcolorbar=False, plot_edges=None):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangulation;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data
    
    points3D=np.vstack((x,y,z)).T
    
    tri_vertices= points3D[simplices]# vertices of the surface triangles  
   
    zmean=tri_vertices[:, :, 2].mean(-1)# mean values of z-coordinates of the
                                        #triangle vertices
    
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)  
   
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean] 
    I,J,K=zip(*simplices)
    
    triangles=Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor, 
                     i=I,
                     j=J,
                     k=K,
                     name=''
                    )
    if showcolorbar==True:
        pl_colorsc=mpl_to_plotly(colormap,11)
        # a fake Scatter3d defined in order to display the colorbar for the trisurf
        colorbar = Scatter3d(x=tri_vertices[:,0,0],
                             y=tri_vertices[:,0,1],
                             z=tri_vertices[:,0,2],
                             mode='markers',
                             marker=dict(size=0.1, color=zmean, colorscale=pl_colorsc, showscale=True),
                             hoverinfo='None')

    
    
    if plot_edges is None: # the triangle sides are not plotted 
        if  showcolorbar is True:
            return Data([colorbar, triangles])
        else: 
            return  Data([triangles])
    else:#plot edges
        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        Xe=[]
        Ye=[]
        Ze=[]
        for T in tri_vertices:
            Xe+=[T[k%3][0] for k in range(4)]+[ None]
            Ye+=[T[k%3][1] for k in range(4)]+[ None]
            Ze+=[T[k%3][2] for k in range(4)]+[ None]
       
        #define the lines to be plotted
        lines=Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=Line(color= 'rgb(50,50,50)', width=1.5)
               )
        if  showcolorbar is True:
            return Data([colorbar, triangles, lines])
        else: 
            
            return Data([triangles, lines])

We plot the surface of equation $\cos(x)+\cos(y)+\cos(z)=0$, $x,y,z \in [0,1]$:

In [49]:
X,Y,Z=2*np.pi*np.mgrid[-1:1:41j, -1:1:41j, -1:1:41j]

surf_eq=np.cos(X) +np.cos(Y)+np.cos(Z)
verts, faces = measure.marching_cubes(surf_eq, 0, spacing=(0.1, 0.1, 0.1))
title='Schwartz P surface' 
cmap=cm.RdBu_r 

In [50]:
x,y,z=zip(*verts)

In [51]:
data=plotly_trisurf(x,y,z, faces, colormap=cmap, showcolorbar=True)

In [52]:
axis = dict(
showbackground=True, 
backgroundcolor="rgb(230, 230,230)",
gridcolor="rgb(255, 255, 255)",      
zerolinecolor="rgb(255, 255, 255)",  
    )

noaxis=dict( showbackground=False,
            showgrid=False,
            showline=False,
            showticklabels=False,
            ticks='',
            title='',
            zeroline=False)

layout = Layout(
         title=title, 
         showlegend=False,
         width=800,
         height=800,
         scene=Scene(xaxis=XAxis(axis),
                     yaxis=YAxis(axis), 
                     zaxis=ZAxis(axis), 
                     aspectratio=dict(x=1,
                                      y=1, 
                                      z=1
                                     ),
                    )
        )



In [53]:
fig = Figure(data=data, layout=layout)

In [54]:

plotly.offline.iplot(fig)

In [55]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()